In [106]:
import os 
from pathlib import Path 

import numpy as np
import pandas as pd

import xarray as xr
import netCDF4 as nc
import wrf
import xwrf

import matplotlib.pyplot as plt


# Functions

In [219]:
def format_times(da):
    ts = pd.to_datetime(da.time.values)
    tsPDT = ts - pd.DateOffset(hours=7)
    da['time'] = tsPDT
    return da

def plot_vprof(df, outfpath=None, display_plot=True):
    fig, ax = plt.subplots(figsize=(10, 12))
    ax.plot(df['relative humidity'],
            df['height_agl'])
    ax.set(title='RH vs. Height A.G.L.',
        xlabel='Relative Humidity (%)',
        ylabel='Height A.G.L. (m)')
    
    ax.set_xticks([0, 20, 40, 60, 80, 100])  # Set consistent x-axis tick marks
    ax.set_xlim(0, 100)  # Set consistent x-axis limits for comparability
    ax.set_yticks([0, 500, 1000, 1500, 2000, 2500, 3000, 3500, 4000, 4500, 5000])  # Set consistent x-axis tick marks
    ax.set_ylim(0, 5000)  # Set consistent x-axis limits for comparability
    
    if outfpath:
        plt.savefig(outfpath)
    
    if display_plot == True:
        plt.show()
    else:
        plt.close()

# Main

In [107]:
# Load in data
root_dir = Path().resolve().parent
data_dir = root_dir / 'data' / 'geospatial' / 'wrf-vertical-profile'
fname = 'wrfout-2008-08-vertprofile.nc'
fpath = str(data_dir / fname)
ncfile = nc.Dataset(data_dir / fname)

In [108]:
hgt_agl = wrf.getvar(ncfile, 'height_agl')
hgt_agl

lats = hgt_agl['XLAT'].values[:, 0]
lons = hgt_agl['XLONG'].values[0, :]
bottom_top = hgt_agl['bottom_top'].values
values = hgt_agl.values

hgt_da = xr.DataArray(
    data = values,
    dims=['bottom_top', 'x', 'y'],
    coords = {'bottom_top' : bottom_top, 'y' : lons, 'x' : lats}
    )

hgt_da

<xarray.DataArray (bottom_top: 54, x: 234, y: 258)> Size: 13MB
array([[[   25.208294,    25.207727,    25.206139, ...,    25.576002,
            25.575455,    25.57454 ],
        [   25.209743,    25.208824,    25.206333, ...,    25.577946,
            25.578644,    25.578806],
        [   25.208776,    25.207153,    25.204971, ...,    25.576778,
            25.57772 ,    25.578812],
        ...,
        [   25.132458,    25.132198,    25.133635, ...,    25.576782,
            25.593018,    25.634033],
        [   25.13129 ,    25.131334,    25.133053, ...,    25.713745,
            25.715454,    25.726807],
        [   25.1305  ,    25.131289,    25.132643, ...,    25.866455,
            25.848755,    25.821777]],

       [[   82.61463 ,    82.61304 ,    82.60783 , ...,    83.81501 ,
            83.81292 ,    83.80956 ],
        [   82.61933 ,    82.615746,    82.608185, ...,    83.821175,
            83.82349 ,    83.82408 ],
        [   82.616005,    82.610245,    82.603134, ...,    83.817375,
            83.82054 ,    83.824104],
...
        [20272.85    , 20272.89    , 20272.91    , ..., 18608.633   ,
         18616.24    , 18648.057   ],
        [20272.918   , 20272.97    , 20272.986   , ..., 18748.848   ,
         18746.428   , 18750.14    ],
        [20272.984   , 20273.002   , 20273.02    , ..., 18902.625   ,
         18880.705   , 18851.357   ]],

       [[20659.58    , 20659.592   , 20659.607   , ..., 20660.412   ,
         20659.178   , 20658.059   ],
        [20659.676   , 20659.7     , 20659.709   , ..., 20663.041   ,
         20663.188   , 20663.174   ],
        [20659.773   , 20659.795   , 20659.807   , ..., 20663.299   ,
         20663.365   , 20663.363   ],
        ...,
        [20678.846   , 20678.895   , 20678.914   , ..., 18939.57    ,
         18947.492   , 18980.564   ],
        [20678.926   , 20678.984   , 20679.002   , ..., 19085.516   ,
         19082.984   , 19086.832   ],
        [20679.004   , 20679.023   , 20679.043   , ..., 19245.584   ,
         19222.754   , 19192.195   ]]], dtype=float32)
Coordinates:
  * bottom_top  (bottom_top) int64 432B 0 1 2 3 4 5 6 7 ... 47 48 49 50 51 52 53
  * y           (y) float32 1kB -121.3 -121.2 -121.2 ... -118.6 -118.6 -118.6
  * x           (x) float32 936B 33.51 33.52 33.53 33.53 ... 35.49 35.49 35.5

In [109]:
ds = wrf.getvar(ncfile, 'rh', timeidx=wrf.ALL_TIMES)
# Select based on coordinates
sauclat = 34.001033
sauclon = -119.817817

In [110]:
lats = ds['XLAT'].values[:, 0]
lons = ds['XLONG'].values[0, :]
bottom_top = ds['bottom_top'].values
times = ds.Time.values
rh2 = ds.values

da = xr.DataArray(
    data = rh2,
    dims=['time', 'bottom_top', 'x', 'y'],
    coords = {'time' : times, 'bottom_top' : bottom_top, 'y' : lons, 'x' : lats}
)

<xarray.DataArray (time: 96, bottom_top: 54, x: 234, y: 258)> Size: 1GB
array([[[[ 90.26697   ,  90.07767   ,  89.89238   , ...,  79.03064   ,
           78.96646   ,  78.94257   ],
         [ 90.24443   ,  90.07859   ,  89.901566  , ...,  78.79643   ,
           78.61947   ,  78.50221   ],
         [ 90.09733   ,  89.93876   ,  89.797104  , ...,  78.87693   ,
           78.656075  ,  78.44999   ],
         ...,
         [ 90.18323   ,  90.20013   ,  90.181595  , ...,  20.229273  ,
           20.126886  ,  19.652409  ],
         [ 90.0849    ,  90.08261   ,  90.05366   , ...,  18.952358  ,
           18.979513  ,  18.847006  ],
         [ 89.96545   ,  89.92849   ,  89.91048   , ...,  17.529854  ,
           17.761166  ,  17.979704  ]],

        [[ 92.70477   ,  92.50023   ,  92.30188   , ...,  80.29795   ,
           80.233376  ,  80.23763   ],
         [ 92.69108   ,  92.49871   ,  92.29638   , ...,  79.9967    ,
           79.81255   ,  79.75607   ],
         [ 92.5226    ,  92.342896  ,  92.17787   , ...,  80.08389   ,
           79.87189   ,  79.80691   ],
...
         [  1.3984996 ,   1.3982494 ,   1.3977139 , ...,   1.300031  ,
            1.2986652 ,   1.3012648 ],
         [  1.396173  ,   1.3960992 ,   1.395716  , ...,   1.3056033 ,
            1.3048631 ,   1.3048927 ],
         [  1.393455  ,   1.3934903 ,   1.3931314 , ...,   1.3135263 ,
            1.3123353 ,   1.3098243 ]],

        [[  1.2810231 ,   1.2786819 ,   1.2760485 , ...,   1.2279774 ,
            1.2301174 ,   1.2322713 ],
         [  1.279615  ,   1.2772778 ,   1.2746416 , ...,   1.2288796 ,
            1.2306881 ,   1.2325413 ],
         [  1.2781172 ,   1.2757965 ,   1.2733716 , ...,   1.2284964 ,
            1.2303237 ,   1.2322404 ],
         ...,
         [  1.0780473 ,   1.0790762 ,   1.0801624 , ...,   1.092793  ,
            1.0910361 ,   1.0935297 ],
         [  1.076475  ,   1.0775537 ,   1.0787444 , ...,   1.0921731 ,
            1.0917916 ,   1.092472  ],
         [  1.075517  ,   1.0766088 ,   1.0777701 , ...,   1.0929688 ,
            1.0930911 ,   1.0924631 ]]]], dtype=float32)
Coordinates:
  * time        (time) datetime64[ns] 768B 2008-08-17 ... 2008-08-25T23:00:00
  * bottom_top  (bottom_top) int64 432B 0 1 2 3 4 5 6 7 ... 47 48 49 50 51 52 53
  * y           (y) float32 1kB -121.3 -121.2 -121.2 ... -118.6 -118.6 -118.6
  * x           (x) float32 936B 33.51 33.52 33.53 33.53 ... 35.49 35.49 35.5

In [197]:
sauc_da = da.sel(x=[sauclat], y=[sauclon], method='nearest').squeeze()
sauc_da = format_times(sauc_da)
sauc_heights = hgt_da.sel(x=[sauclat], y=[sauclon], method='nearest').squeeze().values

In [198]:
sauc_da.time

<xarray.DataArray 'time' (time: 96)> Size: 768B
array(['2008-08-16T17:00:00.000000000', '2008-08-16T18:00:00.000000000',
       '2008-08-16T19:00:00.000000000', '2008-08-16T20:00:00.000000000',
       '2008-08-16T21:00:00.000000000', '2008-08-16T22:00:00.000000000',
       '2008-08-16T23:00:00.000000000', '2008-08-17T00:00:00.000000000',
       '2008-08-17T01:00:00.000000000', '2008-08-17T02:00:00.000000000',
       '2008-08-17T03:00:00.000000000', '2008-08-17T04:00:00.000000000',
       '2008-08-17T05:00:00.000000000', '2008-08-17T06:00:00.000000000',
       '2008-08-17T07:00:00.000000000', '2008-08-17T08:00:00.000000000',
       '2008-08-17T09:00:00.000000000', '2008-08-17T10:00:00.000000000',
       '2008-08-17T11:00:00.000000000', '2008-08-17T12:00:00.000000000',
       '2008-08-17T13:00:00.000000000', '2008-08-17T14:00:00.000000000',
       '2008-08-17T15:00:00.000000000', '2008-08-17T16:00:00.000000000',
       '2008-08-17T17:00:00.000000000', '2008-08-17T18:00:00.000000000',
       '2008-08-17T19:00:00.000000000', '2008-08-17T20:00:00.000000000',
       '2008-08-17T21:00:00.000000000', '2008-08-17T22:00:00.000000000',
       '2008-08-17T23:00:00.000000000', '2008-08-18T00:00:00.000000000',
       '2008-08-18T01:00:00.000000000', '2008-08-18T02:00:00.000000000',
       '2008-08-18T03:00:00.000000000', '2008-08-18T04:00:00.000000000',
       '2008-08-18T05:00:00.000000000', '2008-08-18T06:00:00.000000000',
       '2008-08-18T07:00:00.000000000', '2008-08-18T08:00:00.000000000',
       '2008-08-18T09:00:00.000000000', '2008-08-18T10:00:00.000000000',
       '2008-08-18T11:00:00.000000000', '2008-08-18T12:00:00.000000000',
       '2008-08-18T13:00:00.000000000', '2008-08-18T14:00:00.000000000',
       '2008-08-18T15:00:00.000000000', '2008-08-18T16:00:00.000000000',
       '2008-08-23T17:00:00.000000000', '2008-08-23T18:00:00.000000000',
       '2008-08-23T19:00:00.000000000', '2008-08-23T20:00:00.000000000',
       '2008-08-23T21:00:00.000000000', '2008-08-23T22:00:00.000000000',
       '2008-08-23T23:00:00.000000000', '2008-08-24T00:00:00.000000000',
       '2008-08-24T01:00:00.000000000', '2008-08-24T02:00:00.000000000',
       '2008-08-24T03:00:00.000000000', '2008-08-24T04:00:00.000000000',
       '2008-08-24T05:00:00.000000000', '2008-08-24T06:00:00.000000000',
       '2008-08-24T07:00:00.000000000', '2008-08-24T08:00:00.000000000',
       '2008-08-24T09:00:00.000000000', '2008-08-24T10:00:00.000000000',
       '2008-08-24T11:00:00.000000000', '2008-08-24T12:00:00.000000000',
       '2008-08-24T13:00:00.000000000', '2008-08-24T14:00:00.000000000',
       '2008-08-24T15:00:00.000000000', '2008-08-24T16:00:00.000000000',
       '2008-08-24T17:00:00.000000000', '2008-08-24T18:00:00.000000000',
       '2008-08-24T19:00:00.000000000', '2008-08-24T20:00:00.000000000',
       '2008-08-24T21:00:00.000000000', '2008-08-24T22:00:00.000000000',
       '2008-08-24T23:00:00.000000000', '2008-08-25T00:00:00.000000000',
       '2008-08-25T01:00:00.000000000', '2008-08-25T02:00:00.000000000',
       '2008-08-25T03:00:00.000000000', '2008-08-25T04:00:00.000000000',
       '2008-08-25T05:00:00.000000000', '2008-08-25T06:00:00.000000000',
       '2008-08-25T07:00:00.000000000', '2008-08-25T08:00:00.000000000',
       '2008-08-25T09:00:00.000000000', '2008-08-25T10:00:00.000000000',
       '2008-08-25T11:00:00.000000000', '2008-08-25T12:00:00.000000000',
       '2008-08-25T13:00:00.000000000', '2008-08-25T14:00:00.000000000',
       '2008-08-25T15:00:00.000000000', '2008-08-25T16:00:00.000000000'],
      dtype='datetime64[ns]')
Coordinates:
    y        float32 4B -119.8
    x        float32 4B 34.0
  * time     (time) datetime64[ns] 768B 2008-08-16T17:00:00 ... 2008-08-25T16...

In [220]:
dateranges = [['2008-08-16 17:00:00','2008-08-18 16:00:00'], 
              ['2008-08-23 17:00:00','2008-08-25 16:00:00']]

for daterange in dateranges:
    dates = [date.split(' ')[0] for date in daterange]
    months = [date.split('-')[1] for date in dates]
    days = [date.split('-')[2] for date in dates]


    outdir_root = '/Users/patmccornack/Documents/ucsb_fog_project/_repositories/sci-wrf-analysis/outputs/spatial-analysis-figs/vertical-profiles/'
    outdir_name = f'rh-vertp-{months[0]}-{days[0]}_{months[1]}-{days[1]}/'

    if not os.path.isdir(outdir_root + outdir_name):
        os.mkdir(outdir_root + outdir_name)

    time_range = pd.date_range(start=daterange[0], end=daterange[1], freq='h')
    i = 1
    for timestamp in time_range:
        outfpath = os.path.join(outdir_root + outdir_name, f'{i}-sauc-{timestamp}.png')
        rh_values = sauc_da.sel(time=[timestamp]).values.squeeze()
        df = pd.DataFrame({'height_agl' : sauc_heights,
                        'relative humidity' : rh_values})
        plot_vprof(df, outfpath, display_plot=False)
        i+=1

    print(outfpath)

/Users/patmccornack/Documents/ucsb_fog_project/_repositories/sci-wrf-analysis/outputs/spatial-analysis-figs/vertical-profiles/rh-vertp-08-16_08-18/48-sauc-2008-08-18 16:00:00.png
/Users/patmccornack/Documents/ucsb_fog_project/_repositories/sci-wrf-analysis/outputs/spatial-analysis-figs/vertical-profiles/rh-vertp-08-23_08-25/48-sauc-2008-08-25 16:00:00.png
